In [1]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq transformers==4.31.0 --progress-bar off
!pip install -qqq langchain==0.0.266 --progress-bar off
!pip install -qqq chromadb==0.4.5 --progress-bar off
!pip install -qqq pypdf==3.15.0 --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off

^C
ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: none)
ERROR: No matching distribution found for torch==2.0.1


In [ ]:
!pip install googletrans==3.1.0a0 --progress-bar off

In [ ]:
from googletrans import Translator
translator = Translator()

In [ ]:
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl

In [ ]:
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off

In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
!mkdir pdfs

In [ ]:
!rm -rf "/kaggle/working/db1"

In [ ]:
# Load constituion of India
loader = PyPDFDirectoryLoader("/kaggle/input/lawgpt-pdfs/Constitutuion of India")
docs = loader.load()
len(docs)

#load labour laws
loader = PyPDFDirectoryLoader("/kaggle/input/lawgpt-pdfs/Labour_Laws")
docs1 = loader.load()
len(docs1)

#load /kaggle/input/lawgpt-pdfs/Family_Laws
loader = PyPDFDirectoryLoader("/kaggle/input/lawgpt-pdfs/Family_Laws")
docs2 = loader.load()
len(docs2)

#load /kaggle/input/lawgpt-pdfs/MotorVehicle_Laws
loader = PyPDFDirectoryLoader("/kaggle/input/lawgpt-pdfs/MotorVehicle_Laws")
docs3 = loader.load()
len(docs3)


#load /kaggle/input/lawgpt-pdfs/Property_Laws
loader = PyPDFDirectoryLoader("/kaggle/input/lawgpt-pdfs/Property_Laws")
docs4 = loader.load()
len(docs4)


#load /kaggle/input/lawgpt-pdfs/Criminal_Laws
loader = PyPDFDirectoryLoader("/kaggle/input/lawgpt-pdfs/Criminal_Laws")
docs5 = loader.load()
len(docs5)

In [ ]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)

# constitution of India
texts = text_splitter.split_documents(docs)

#labour laws
texts1 = text_splitter.split_documents(docs1)

#Family_Laws
texts2 = text_splitter.split_documents(docs2)

#MotorVehicle_Laws
texts3 = text_splitter.split_documents(docs3)

#Property_Laws
texts4 = text_splitter.split_documents(docs4)

#Criminal_Laws
texts5 = text_splitter.split_documents(docs5)

print(len(texts), len(texts1))

In [ ]:
%%time
# constitution of India
db = Chroma.from_documents(texts, embeddings, persist_directory="db")
#labour laws
db1 = Chroma.from_documents(texts1, embeddings, persist_directory="db1")
#Family_Laws
db2 = Chroma.from_documents(texts2, embeddings, persist_directory="db2")
#MotorVehicle_Laws
db3 = Chroma.from_documents(texts3, embeddings, persist_directory="db3")
#Property_Laws
db4 = Chroma.from_documents(texts4, embeddings, persist_directory="db4")
#Criminal_Laws
db5 = Chroma.from_documents(texts5, embeddings, persist_directory="db5")

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

In [ ]:
!nvidia-smi

In [ ]:

DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


In [ ]:

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


In [ ]:

SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
# constitution of india
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

# Labour laws
qa_chain1 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db1.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

#Family_Laws
qa_chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db2.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

#MotorVehicle_Laws
qa_chain3 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db3.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

#Property_Laws
qa_chain4 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db4.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

#Criminal_Laws
qa_chain5 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db5.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [ ]:
result = qa_chain("can a 14 year old be emplyeed?")

print(result['result'])

In [ ]:
len(result["source_documents"])

In [ ]:
print(result["source_documents"][0].page_content)

In [ ]:
result = qa_chain("What is the per share revenue for Tesla during 2023?")

In [ ]:
result = qa_chain1("what is Apprentices Act, 1961?")

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()

# middlewares
app.add_middleware(
    CORSMiddleware, # https://fastapi.tiangolo.com/tutorial/cors/
    allow_origins=['*'], # wildcard to allow all, more here - https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Origin
    allow_credentials=True, # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Credentials
    allow_methods=['*'], # http|s://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Methods
    allow_headers=['*'], # https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/Access-Control-Allow-Headers
)

In [ ]:
# google trans demo
query_lang = translator.detect('संविधानाने नागरिकांना कोणता अधिकार दिला आहे?').lang
print("language is ", query_lang)

if query_lang != 'en':
        query_translate = translator.translate('संविधानाने नागरिकांना कोणता अधिकार दिला आहे?').text  # Translate to English
        print(query_translate)

In [ ]:
@app.get("/")
async def llmQuery(query: str, type_law: str):
    query_lang = translator.detect(query).lang
    print(query_lang, type_law)
    
    query_translate = query  # Initialize query_translate
    print(query_translate)
    
    if query_lang != 'en':
        query_translate = translator.translate(query).text  # Translate to English
        print(query_translate)
    
    print("query is", query_translate)
    
    res = {}
    if type_law == "Know Your Rights":
        res = qa_chain(query_translate)
    
    if type_law == "Labour Laws":
        res = qa_chain1(query_translate)
    
    if type_law == "Family Laws":
        res = qa_chain2(query_translate)
    
    if type_law == "Vehicle Laws":
        res = qa_chain3(query_translate)
    
    if type_law == "Property Laws":
        res = qa_chain4(query_translate)
    
    if type_law == "Criminal Laws":
        res = qa_chain5(query_translate)
        
    print(res)
    
    res["regional_res"] = res['result']
    if query_lang != 'en':
        translation = translator.translate(res['result'], src='en', dest=query_lang)
        res["regional_res"] = translation.text
    
    print(res["regional_res"])
    
    return {"result" : res}

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn


# specify a port
port = 8000
ngrok_tunnel = ngrok.connect(port)

# where we can visit our fastAPI app
print('Public URL:', ngrok_tunnel.public_url)


nest_asyncio.apply()

# finally run the app
uvicorn.run(app, port=port)